In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from functools import partial
import gridsearch
import matplotlib.pyplot as plt
from dbn.models import SupervisedDBNRegression
from elm.elm import Model as elm
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.base import clone
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import ParameterGrid
import sklearn.metrics as metrics
from sklearn import preprocessing
from statsmodels.tsa.stattools import adfuller
from multiprocessing import Pool
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

In [2]:
file_names = ["amz", "APPLE", "electricity", "goldman", "msft", "pollutions", "star", "sunspot", "vehicle", "wine"]
file_prefix = "dataset/"
file_suffix = ".txt"
models = [
          {'model_name': "SVR", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "GB", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "RF", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "ARIMA", 'windows': False, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "DBN", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "MLP", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
          {'model_name': "ELM", 'windows': True, 'lags': 20, 'normalize_before_metrics': True, 'differencing_order': 'ADF', 'level': '5%', 'p_level': 0.05},
         ]


random_seed = 156

execution = "dsnaw" # "oracle" or "dsnaw"
plot_curves = False
train_proportion = 1/2
val_proportion = 1/4
max_k = 20
max_n = len(models)

In [3]:
#read files
def read_file(path):
    array_ts = np.loadtxt(path)
    df = pd.DataFrame(data = array_ts, columns = ['y'])
    return df

In [4]:
#define split points

def define_split_points(df, train, val):
    val_index = int(df.shape[0]*train)
    test_index = int(df.shape[0]*(train+val))

    # fig, ax=plt.subplots(figsize=(9, 4))
    # df['y'].loc[:val_index-1].plot(ax=ax, label = 'train')
    # df['y'].loc[val_index:test_index-1].plot(ax=ax, label = 'validation')
    # df['y'].loc[test_index:].plot(ax=ax, label = 'test')
    # ax.legend()
    
    return val_index, test_index


In [5]:
#difference time series

def check_ADF(ts, lvl, p_lvl):
    result = adfuller(ts)
    return result[0] < result[4][lvl] and result[1] < p_lvl

def difference(df, differencing_order, level, p_level):

    df_differenced = None
    if differencing_order:    
        df_differenced = df.copy()

        if differencing_order == 'ADF':
            stationary = check_ADF(df_differenced['y'].values, level, p_level)
            d = 0
            while not stationary:
                df_differenced = df_differenced.diff().dropna()
                d += 1
                stationary = check_ADF(df_differenced['y'].values, level, p_level)
            if d == 0:
                df_differenced = None
            differencing_order = d
        else:
            i = 0
            while i < differencing_order:
                df_differenced = df_differenced.diff().dropna()
                i+=1
                
    print("d: ", differencing_order)
    # if differencing_order:
    #     fig, ax=plt.subplots(figsize=(9, 4))
    #     df_differenced.y.plot(ax = ax)
        
    return df_differenced, differencing_order


In [6]:
#normalize time series

def normalize(df, val_index, d):
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler.fit(df['y'].loc[:val_index-1].values.reshape(-1,1))
    df_normalized = pd.DataFrame({'y': np.append(np.zeros(d),min_max_scaler.transform(df['y'].values.reshape(-1, 1)).flatten())}).iloc[d:]
    
    return df_normalized, min_max_scaler


In [7]:
#create windows

def create_windows(df, lags):
    df_windowed = df.copy()
    df_windowed['x'] = df_windowed['y'].shift()
    for i in range(1, lags):
        df_windowed[f'x-{i}'] = df_windowed['x'].shift(i)
    df_windowed = df_windowed.dropna()
    return df_windowed

In [8]:
#gridsearch

def parallel_gridsearch(estimator, param_grid, x_train, y_train, x_val, y_val):
    pool = Pool()
    list_params = list(ParameterGrid(param_grid))
    results = list(tqdm(pool.imap(partial(gridsearch.validate_params, estimator=estimator, x_train=x_train, 
                               y_train=y_train, x_val=x_val, y_val=y_val), list_params), total=len(list_params)))
    idx = np.argmin([r[1] for r in results])
    best_params = results[idx][0]
    best_rmse = results[idx][1]
    
    pool.close()
    pool.join()
    print('Best params: ', best_params)
    print('Best rmse: ', best_rmse)
    return best_params
    

In [9]:
#SVR

def svr_model(lags):
    parameters = {'C':[1, 10, 100, 1000], 'gamma': [0.001, 0.01, 0.1, 1],
                      'kernel':["rbf"],
                      'epsilon': [0.1, 0.01, 0.001, 0.0001, 0.00001],
                 }

    model = SVR(max_iter = 10000)

    return model, parameters

In [10]:
#Gradient Boosting

def gb_model(lags):
    parameters = {'n_estimators': [50, 100, 200], 
                  'max_depth': [5, 10, 15],
                  'max_features': [0.6, 0.8, 1],
                  'subsample' : [0.6, 0.8, 1],
                  'learning_rate': [0.1, 0.3, 0.5],
                 }
    
    model = GradientBoostingRegressor(random_state=random_seed)

    return model, parameters

In [11]:
#Random Forest

def rf_model(lags):
    parameters = {'n_estimators': [50, 100, 200], 
                  'max_depth': [5, 10, 15],
                  'max_features': [0.6, 0.8, 1],
                 }
    
    model = RandomForestRegressor(random_state=random_seed)

    return model, parameters

In [12]:
#MLP

def mlp_model(lags):
    parameters = {'hidden_layer_sizes': [20, 50, 100], 
                      'max_iter': [1000],
                      'tol': [0.001, 0.0001, 0.00001],
                 }
    
    model = MLPRegressor(activation='logistic', solver='lbfgs', random_state=random_seed)
    
    return model, parameters

In [13]:
#ELM

def elm_model(lags):
    parameters = {'hidden_dim': [20, 50, 100, 200, 500],  
                 }
    
    model = elm(input_dim=lags)
    
    return model, parameters

In [14]:
#Deep Belief Network

def dbn_model(lags):
    
    model = SupervisedDBNRegression(hidden_layers_structure=[100],
                                    learning_rate_rbm=0.01,
                                    learning_rate=0.01,
                                    n_epochs_rbm=20,
                                    n_iter_backprop=200,
                                    batch_size=16,
                                    activation_function='relu',
                                    verbose=False)

    return model

In [15]:
models_name = {
    'SVR': svr_model, 
    'GB': gb_model,
    'RF': rf_model,
    'DBN': dbn_model,
    'MLP': mlp_model,
    'ELM': elm_model,
         }

In [16]:
#train model
@ignore_warnings(category=ConvergenceWarning)
def train_model(model_name, df, val_index, test_index, lags):

    if model_name == "ARIMA":
        
        model = pm.auto_arima(df['y'].loc[:test_index-1],
                          test='adf',       
                          max_p=lags, max_q=lags,        
                          seasonal=False,   
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)
        best_params=model.get_params()
        
    elif model_name == "DBN":
        
        model = dbn_model(lags)
        
        x_train = df.drop(columns = ['y']).loc[:test_index-1].values
        y_train = df['y'].loc[:test_index-1].values
        
        model.fit(x_train, y_train)
        best_params={}
        
    else:
        
        model, param_grid = models_name[model_name](lags)

        x_train = df.drop(columns = ['y']).loc[:val_index-1].values
        y_train = df['y'].loc[:val_index-1].values
        x_val = df.drop(columns = ['y']).loc[val_index:test_index-1].values
        y_val = df['y'].loc[val_index:test_index-1].values
        
        
        best_params = parallel_gridsearch(model, param_grid, 
                                    x_train, y_train,
                                    x_val, y_val)

        model.set_params(**best_params)
        model.fit(x_train, y_train)

        
    
    return model, best_params

In [17]:
#test model

def test_model(model, model_name, df, test_index, scaler):
    if model_name != "ARIMA":
        y_test = df['y'].loc[test_index:].values
        x_test = df.drop(columns = ['y']).loc[test_index:].values

        y_pred = model.predict(x_test)

    else:
        y_test = df['y'].loc[test_index:].values
        y_pred = np.zeros(y_test.size)
        for i in tqdm(range(y_pred.size)):
            y_pred[i] = model.predict(n_periods = 1)
            model.update([y_test[i]])

    y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    return y_pred, y_test

In [18]:
def recursive_differencing_coefficients(order):
    if order <= 1:
        return np.array([1, -1])
    else:
        prev_coefficients = recursive_differencing_coefficients(order-1)
        coefficients = np.zeros(order+1)
        coefficients[0] = prev_coefficients[0]
        coefficients[-1] = -prev_coefficients[-1]
        for i in range(1, order):
            coefficients[i] = prev_coefficients[i] - prev_coefficients[i-1]
        return coefficients   

def inverse_difference(differencing_order, df, df_diff, test_index, y_pred):
    df_pred = df.copy()
    
    for i in range(1, differencing_order + 1):
        df_pred[f'y-{i}'] = df_pred['y'].shift(i)
    
    df_pred['y_diff'] = df_diff['y']
    df_pred['y_diff'][test_index:] = y_pred
    
    df_pred.fillna(0, inplace=True)
    
    coefficients = recursive_differencing_coefficients(differencing_order)
    df_pred['y_pred'] = df_pred['y_diff']
    for i in range(1, differencing_order + 1):
        df_pred['y_pred'] = df_pred['y_pred'] - coefficients[i]*df_pred[f'y-{i}']
        
    y_pred = df_pred['y_pred'][test_index:].values    
    y_test = df_pred['y'][test_index:].values
    
    return y_pred, y_test


In [19]:
#plot results
def plot_results(y_pred, y_test):
    fig, ax=plt.subplots(figsize=(9, 4))
    plt.plot(y_pred, label = 'predicted')
    plt.plot(y_test, label = 'real')
    ax.legend()
    plt.show()

In [20]:
def oracle(preds, real):
    best_pred = np.zeros(real.size)
    for i in range(real.size):
        candidates = np.array([p[i] for p in preds])
        idx = np.argmin(np.absolute(candidates - real[i]))
        best_pred[i] = candidates[idx]
    return best_pred

In [21]:
#calculate metrics

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    posi_with_zeros = np.where(y_true == 0)[0]

    y_true = [n for k, n in enumerate(y_true) if k not in posi_with_zeros]
    y_pred = [n for k, n in enumerate(y_pred) if k not in posi_with_zeros]
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    posi_with_zeros = np.where(y_true == 0)[0]

    y_true = [n for k, n in enumerate(y_true) if k not in posi_with_zeros]
    y_pred = [n for k, n in enumerate(y_pred) if k not in posi_with_zeros]
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    posi_with_zeros = np.where((np.abs(y_true)+np.abs(y_pred))/2 == 0 )[0]

    y_true = [n for k, n in enumerate(y_true) if k not in posi_with_zeros]
    y_pred = [n for k, n in enumerate(y_pred) if k not in posi_with_zeros]
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true)+np.abs(y_pred))/2)) * 100


def mean_absolute_error(y_true, y_pred):
    
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    return np.mean(np.abs(y_true - y_pred))

def average_relative_variance(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    mean = np.mean(y_true)

    error_sup = np.square(np.subtract(y_true, y_pred)).sum()
    error_inf = np.square(np.subtract(y_pred, mean)).sum()

    return error_sup / error_inf

def calculate_metrics(y_pred, y_test, normalize_before_metrics, scaler, df):
    if normalize_before_metrics:
        scaler.fit(df['y'].values.reshape(-1,1))
        y_pred = scaler.transform(y_pred.reshape(-1, 1)).flatten()
        y_test = scaler.transform(y_test.reshape(-1, 1)).flatten()

    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    nrmse = rmse/(y_test.max()-y_test.min())
    mape = mean_absolute_percentage_error(y_test, y_pred)
    smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)
    arv = average_relative_variance(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    res = {'MSE': mse, 'MAPE': mape, 'ARV': arv, 'MAE': mae, 'RMSE': rmse, 'NRMSE': nrmse, 'SMAPE': smape}
    
    for key, value in res.items():
        print(f'{key}: {value}')
    
    return res

In [22]:
#gridsearch

def parallel_gridsearch_dsnaw(df, pred_results, val_index, test_index, param_grid):
    pool = Pool()
    list_params = list(ParameterGrid(param_grid))
    results = list(tqdm(pool.imap(partial(gridsearch.validate_dsnaw_params, df=df, pred_results=pred_results, val_index=val_index, test_index=test_index), list_params), total=len(list_params)))
    idx = np.argmin([r[1] for r in results])
    
    best_params = results[idx][0]
    best_rmse = results[idx][1]
    
    pool.close()
    pool.join()
    
    print('Best params: ', best_params)
    print('Best rmse: ', best_rmse)
    return best_params

In [23]:
def run_model(execution, df, model_name, train_proportion, val_proportion, windows, lags, max_k, plot_curves, normalize_before_metrics, differencing_order, level, p_level):
    print("\n###############################################\n")
    print("Executing: ", model_name)
    val_index, test_index = define_split_points(df, train_proportion, val_proportion)
    d = 0
    if differencing_order:
        df_differenced, d = difference(df, differencing_order, level, p_level)
    if(d):
        df_normalized, min_max_scaler = normalize(df_differenced, val_index, d)
    else:
        df_normalized, min_max_scaler = normalize(df, val_index, d)
    
    if(windows):
        df_normalized = create_windows(df_normalized, lags)        

    estimator, params = train_model(model_name, df_normalized, val_index, test_index, lags)
    if(execution == "dsnaw"):
        result_index = val_index - max_k
    else:
        result_index = test_index
    y_pred, y_test = test_model(estimator, model_name, df_normalized, result_index, min_max_scaler)
    if d:
        y_pred, y_test = inverse_difference(d, df, df_differenced, result_index, y_pred)        
    if(execution == "dsnaw"):
        if plot_curves:
            plot_results(y_pred[(test_index-result_index):], y_test[(test_index-result_index):])
        m = calculate_metrics(y_pred[(test_index-result_index):], y_test[(test_index-result_index):], normalize_before_metrics, min_max_scaler, df)
    else:
        if plot_curves:
            plot_results(y_pred, y_test)
        m = calculate_metrics(y_pred, y_test, normalize_before_metrics, min_max_scaler, df)
    return y_pred, m

In [24]:
def test_dsnaw(params, df, pred_results, test_index):
    k = params['k']
    n = params['n']
    comb = params['comb']    
    
    y_dsnaw = np.zeros(df['y'].size-test_index)
    for i in range(test_index, df['y'].size):
        rmse_roc = []
        real_roc = df['y'].loc[i-k:i-1].values
        for j in range(0, len(pred_results)):      
            pred_roc = pred_results[j]['y'].loc[i-k:i-1].values
            rmse_roc.append((j, np.sqrt(metrics.mean_squared_error(pred_roc, real_roc))))
        sorted_rmse_roc = sorted(rmse_roc, key=lambda x: x[1])
        
        comb_values = np.zeros(n)
        for j in range(0, n):
            comb_values[j] = pred_results[sorted_rmse_roc[j][0]]['y'].loc[i]
        if comb == 'median':
            y_dsnaw[i-test_index] = np.median(comb_values)
        else:
            y_dsnaw[i-test_index] = np.average(comb_values)

    return y_dsnaw

In [25]:
metric_res = []
params_res = []

for f in file_names:
    print("Time Series: ", f)
    df = read_file(file_prefix + f + file_suffix)

    results = []
    m_results = []
    for args in models:
        run_result = run_model(execution=execution, df=df, train_proportion=train_proportion, val_proportion=val_proportion, plot_curves=plot_curves, max_k = max_k, **args)
        results.append(run_result[0])
        m_results.append((args['model_name'], run_result[1]))

    val_index, test_index = define_split_points(df, train_proportion, val_proportion)
    min_max_scaler = preprocessing.MinMaxScaler()
    real = df['y'].loc[test_index:].values

    if (execution == "oracle"):
        oracle_pred = oracle(results, real)
        calculate_metrics(oracle_pred, real, True, min_max_scaler, df)

    else:
        parameters = {
            'k': list(range(1,max_k +1)),
            'n': list(range(1,max_n +1)),
            'comb': ['median', 'average'],
        }
        grid = list(ParameterGrid(parameters))
        pred_results = []
        for r in results:
            temp = pd.DataFrame(data = np.zeros(df['y'].size), columns = ['y'])
            temp['y'][val_index - max_k:] = r
            pred_results.append(temp)
        
        best_params = parallel_gridsearch_dsnaw(df, pred_results, val_index, test_index, parameters)
        params_res.append((f, best_params))
        y_dsnaw = test_dsnaw(best_params, df, pred_results, test_index)
        m = calculate_metrics(y_dsnaw, real, True, min_max_scaler, df)
        m_results.append(("Proposed", m))
        metric_res.append((f, m_results))
        
    

Time Series:  amz

###############################################

Executing:  SVR
d:  1


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:02<00:00, 30.82it/s]


Best params:  {'C': 1, 'epsilon': 0.01, 'gamma': 0.001, 'kernel': 'rbf'}
Best rmse:  0.08131776087386697
MSE: 0.000116965672799083
MAPE: 14.668850997280291
ARV: 0.032990833288604145
MAE: 0.008399620849528071
RMSE: 0.010815066934563234
NRMSE: 0.043597310346846385
SMAPE: 12.986642290913933

###############################################

Executing:  GB
d:  1


100%|████████████████████████████████████████████████████████████████████████████████| 243/243 [00:13<00:00, 18.01it/s]


Best params:  {'learning_rate': 0.1, 'max_depth': 5, 'max_features': 1, 'n_estimators': 50, 'subsample': 0.8}
Best rmse:  0.0827683777309238
MSE: 0.00012247026688499472
MAPE: 15.593520032938446
ARV: 0.034570539447279634
MAE: 0.008576347690939467
RMSE: 0.011066628523854712
NRMSE: 0.04461139641270663
SMAPE: 13.316431977811183

###############################################

Executing:  RF
d:  1


...

Best params:  {'hidden_layer_sizes': 20, 'max_iter': 1000, 'tol': 0.0001}
Best rmse:  0.07867569191898713
MSE: 0.0014799438012167883
MAPE: 43.951010841363725
ARV: 0.46651897357822913
MAE: 0.0294362217919573
RMSE: 0.03847003770750411
NRMSE: 0.11787614118068568
SMAPE: 36.75818403851831

###############################################

Executing:  ELM
d:  1


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  7.62it/s]


Best params:  {'hidden_dim': 20}
Best rmse:  0.08291752841496676
MSE: 0.0016201848038707055
MAPE: 51.62825753320925
ARV: 0.5211849165840442
MAE: 0.031645151845437404
RMSE: 0.04025151927406847
NRMSE: 0.12333478341669699
SMAPE: 39.78745350433096


100%|████████████████████████████████████████████████████████████████████████████████| 280/280 [00:03<00:00, 88.48it/s]


Best params:  {'comb': 'median', 'k': 16, 'n': 1}
Best rmse:  18.059938909306208
MSE: 0.0017616269702650893
MAPE: 53.252039487872224
ARV: 0.46295284462127057
MAE: 0.03250189605461762
RMSE: 0.04197174013863482
NRMSE: 0.12860571657863748
SMAPE: 39.9414535532061


In [26]:
params_df = pd.DataFrame(columns=['Time Series', 'k', 'n', 'comb'])

for f in params_res:
    new_row = {'Time Series': f[0]}
    for key, value in f[1].items():
        new_row[key] = value
    new_row = pd.Series(new_row)
    params_df = pd.concat([params_df, new_row.to_frame().T], ignore_index=True)
    
    
params_df

,Time Series,k,n,comb
0,amz,3,6,median
1,APPLE,2,5,median
2,electricity,11,6,average
3,goldman,10,1,median
4,msft,18,6,average
5,pollutions,15,4,average
6,star,13,1,median
7,sunspot,1,3,median
8,vehicle,7,2,median
9,wine,16,1,median


In [27]:
columns_models = [m['model_name'] for m in models] + ['Proposed']

metrics_df = pd.DataFrame(columns=['Time Series', 'Metrics'] + columns_models)

metrics_list = list(metric_res[0][1][0][1].keys())

for f in metric_res:
    f_name = f[0]
    rows = {}
    for me in metrics_list:
        rows[me] = []
    for mo in f[1]:
        mo_name = mo[0]
        for me, value in mo[1].items():
            rows[me].append((mo_name, value))
    for me, res in rows.items():
        new_row = {'Time Series': f_name, 'Metrics': me}
        for r in res:
            new_row[r[0]] = r[1]
        new_row = pd.Series(new_row)
        metrics_df = pd.concat([metrics_df, new_row.to_frame().T], ignore_index=True)
        
        
df_dsnaw = pd.read_csv('results/metrics_results_bagging.csv') 

metrics_df['DSNAW'] = df_dsnaw['DSNAW']

columns_models.insert(-1, 'DSNAW')
metrics_df = metrics_df[['Time Series', 'Metrics'] + columns_models]
metrics_df[columns_models] = metrics_df[columns_models].apply(lambda x: pd.to_numeric(x, errors='coerce'))
metrics_df['Best'] = metrics_df[columns_models].idxmin(axis=1)
pd.set_option('display.float_format', '{:.2E}'.format)
print(metrics_df)

   Time Series Metrics      SVR       GB       RF    ARIMA      DBN      MLP  \
0          amz     MSE 1.17E-04 1.22E-04 1.18E-04 1.17E-04 1.17E-04 1.18E-04   
1          amz    MAPE 1.47E+01 1.56E+01 1.47E+01 1.47E+01 1.48E+01 1.45E+01   
2          amz     ARV 3.30E-02 3.46E-02 3.33E-02 3.29E-02 3.29E-02 3.32E-02   
3          amz     MAE 8.40E-03 8.58E-03 8.42E-03 8.42E-03 8.45E-03 8.45E-03   
4          amz    RMSE 1.08E-02 1.11E-02 1.09E-02 1.08E-02 1.08E-02 1.09E-02   
..         ...     ...      ...      ...      ...      ...      ...      ...   
65        wine     ARV 4.45E-01 7.47E-01 1.00E+00 2.65E+00 1.78E+00 4.67E-01   
66        wine     MAE 3.45E-02 4.81E-02 4.25E-02 4.78E-02 5.05E-02 2.94E-02   
67        wine    RMSE 4.11E-02 6.40E-02 5.87E-02 6.68E-02 7.65E-02 3.85E-02   
68        wine   NRMSE 1.26E-01 1.96E-01 1.80E-01 2.05E-01 2.34E-01 1.18E-01   
69        wine   SMAPE 4.16E+01 5.14E+01 4.47E+01 4.70E+01 5.40E+01 3.68E+01   

        ELM    DSNAW  Proposed   Best  

In [28]:
params_df.to_csv('results/params_results.csv', index=False)
metrics_df.to_csv('results/metrics_results.csv', index=False, float_format="%.2E")

In [29]:
metrics_df['Best'].value_counts()

MLP         16
DSNAW       16
Proposed    12
RF           7
SVR          6
DBN          5
ARIMA        5
ELM          2
GB           1
Name: Best, dtype: int64

In [30]:
dsnaw_original = [1.30E-4, 22.40, 3.93E-2, 8.94E-3, 1.14E-2, 4.61E-2, 13.80,
        1.90E-2, 245.72, 2.25, 8.02E-2, 1.38E-1, 3.21E-1, 42.15, 
         3.74E-3,6.933, 2.67E-1, 5.18E-2, 6.11E-2, 1.35E-1, 7.25,
         4.06E-4, 9.52, 2.64E-2, 1.53E-2, 2.01E-2, 4.71E-2, 9.44,
         1.99E-3, 5.03, 1.28E-1, 3.41E-2, 4.46E-2, 9.51E-2, 5.08,
         4.47E-2, 32.26, 1.78, 1.74E-1, 2.11E-1, 4.18E-1, 37.43,
         6.77E-5, 2.99, 1.07E-3, 6.08E-3, 8.23E-3, 8.31E-3, 3.77,
         3.31E-2, 62.44, 9.55E-1, 1.14E-1, 1.82E-1, 1.84E-1, 46.31,
         1.81E-2, 15.39, 1.45, 1.09E-1, 1.34E-1, 2.30E-1, 15.14,
         2.94E-3, 135.02, 1.39, 3.54E-2, 5.43E-2, 2.40E-1, 43.84
        ]
df_dsnaw_2 = pd.DataFrame()
df_dsnaw_2['Time Series'] = metrics_df['Time Series']
df_dsnaw_2['Metrics'] = metrics_df['Metrics']
df_dsnaw_2['DSNAW_ORIGINAL'] = dsnaw_original
df_dsnaw_2['DSNAW'] = metrics_df['DSNAW']

col = ['DSNAW', 'DSNAW_ORIGINAL']

df_dsnaw_2[col] = df_dsnaw_2[col].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_dsnaw_2['Best'] = df_dsnaw_2[col].idxmin(axis=1)
df_dsnaw_2['Best'].value_counts()

DSNAW             61
DSNAW_ORIGINAL     9
Name: Best, dtype: int64

In [31]:
df_direct = pd.DataFrame()
df_direct[['Time Series', 'Metrics', 'DSNAW', 'Proposed']] = metrics_df[['Time Series', 'Metrics', 'DSNAW', 'Proposed']]
col = ['DSNAW', 'Proposed']
df_direct[col] = df_direct[col].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_direct['Best'] = df_direct[col].idxmin(axis=1)
df_direct['Best'].value_counts()

Proposed    46
DSNAW       24
Name: Best, dtype: int64

In [32]:
df_direct.to_csv('results/direct_comparison.csv', index=False, float_format="%.2E")